<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/Inference_using_multivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/dwadden/multivers.git

In [2]:
!gdown https://drive.google.com/uc?id=1KFKGaaiKwx81B_hIeSyvyqiMPDy0e6VZ -O corpus.jsonl
!gdown https://drive.google.com/uc?id=1QwqYmNWQhlsy4Wabg5Lqtv0H1wDjW2N2 -O claims.jsonl

Downloading...
From: https://drive.google.com/uc?id=1KFKGaaiKwx81B_hIeSyvyqiMPDy0e6VZ
To: /content/corpus.jsonl
100% 87.7k/87.7k [00:00<00:00, 87.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QwqYmNWQhlsy4Wabg5Lqtv0H1wDjW2N2
To: /content/claims.jsonl
100% 629/629 [00:00<00:00, 1.43MB/s]


In [3]:
%%capture
!pip install virtualenv
!virtualenv multivers
!source /content/multivers/bin/activate; pip install -r /content/multivers/requirements.txt

In [4]:
!python /content/multivers/script/get_checkpoint.py longformer_large_science
!python /content/multivers/script/get_checkpoint.py scifact


--2023-02-24 14:01:02--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.218.212.1, 52.92.180.82, 3.5.80.155, ...
Connecting to scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)|52.218.212.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646843693 (1.5G) [binary/octet-stream]
Saving to: ‘checkpoints/longformer_large_science.ckpt’

checkpoints/longfor 100%[===================>]   1.53G  4.59MB/s    in 2m 51s  

2023-02-24 14:03:54 (9.18 MB/s) - ‘checkpoints/longformer_large_science.ckpt’ saved [1646843693/1646843693]

--2023-02-24 14:03:54--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/scifact.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.92.131.66, 3.5.78.113, 52.92.146.74, ...
Connecting to scifact.s3.us-west-2.amazona

In [5]:
!mv /content/checkpoints /content/multivers/checkpoints

### If CPU is not available  
delete line `    model.to(f"cuda:{args.device}")` from `predict.py` otherwise it would throw an error `RuntimeError: No CUDA GPUs are available
`

In [6]:
import torch
import os
if not torch.cuda.is_available():
  ret_code = os.system("sed -i '40d' /content/multivers/multivers/predict.py")
  print(f"exit code = {ret_code}")

In [9]:
from timeit import default_timer as timer
start = timer()

In [10]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/scifact.ckpt \
        --batch_size=100 \
        --input_file=/content/claims.jsonl \
        --corpus_file=/content/corpus.jsonl \
        --output_file=/content/preds.jsonl

100% 1/1 [00:15<00:00, 15.12s/it]


In [11]:
end = timer()
print(f"Took {(end - start):.0f} seconds")

Took 80 seconds


In [12]:
!cat /content/preds.jsonl

{"id": 0, "evidence": {}}
{"id": 1, "evidence": {"31": {"label": "CONTRADICT", "sentences": []}, "32": {"label": "CONTRADICT", "sentences": [1]}}}
{"id": 2, "evidence": {}}
